In [24]:
import requests
from bs4 import BeautifulSoup
import locale
from datetime import datetime
import pandas as pd

pd.set_option('display.max_rows', None)

In [3]:
url_fms_arg = 'https://mundofreestyle.com/calendario-fms-argentina/'

In [4]:
response = requests.get(url=url_fms_arg)
response.status_code

200

In [5]:
soup = BeautifulSoup(response.content)

In [6]:
fms_tables_titles = soup.select('h3 strong')
fms_tables = soup.find_all(name='table')

In [7]:
[title.text for title in fms_tables_titles]

['Fechas y ciudades – Temporada 2022',
 'Temporada 2020/2021',
 'Temporada 2019',
 '¡Vive todos los eventos y fechas de FMS Argentina!']

In [8]:
fms_tables[0]

<table style="color: var( --e-global-color-text ); font-family: var( --e-global-typography-text-font-family ), Sans-serif; font-weight: var( --e-global-typography-text-font-weight ); height: 310px;" width="210">
<tbody>
<tr>
<td> </td>
<td><strong>Fecha</strong></td>
<td><strong>Ciudad</strong></td>
</tr>
<tr>
<td>1</td>
<td>4 de diciembre de 2021</td>
<td>Mendoza</td>
</tr>
<tr>
<td>2</td>
<td>29 de diciembre de 2021</td>
<td>Buenos Aires</td>
</tr>
<tr>
<td>3</td>
<td>30 de abril de 2022</td>
<td>La Plata</td>
</tr>
<tr>
<td>4</td>
<td>18 de junio de 2022</td>
<td>Rosario</td>
</tr>
<tr>
<td>5</td>
<td>27 de agosto de 2022</td>
<td>Corrientes</td>
</tr>
<tr>
<td>6</td>
<td></td><td> </td>
</tr>
<tr>
<td>7</td>
<td></td><td> </td>
</tr>
<tr>
<td>8</td>
<td></td><td> </td>
</tr>
<tr>
<td>9</td>
<td></td><td> </td>
</tr>
<tr>
<td>10</td>
<td></td><td> </td>
</tr>
<tr>
<td>11</td>
<td></td><td> </td>
</tr>
</tbody>
</table>

In [77]:
fms_tables[0].select('tr')[2].select('td')[1].text

'29 de diciembre de 2021'

In [78]:
fms_tables[0].select('tr')[0]

<tr>
<td> </td>
<td><strong>Fecha</strong></td>
<td><strong>Ciudad</strong></td>
</tr>

In [79]:
# Testing how to convert a string date into a date type date:

test_date = str(fms_tables[0].select('tr')[2].select('td')[1].text)
test_date_clean = ",".join([word for word in test_date.split(" ") if word != 'de'])

In [80]:
locale.setlocale(locale.LC_ALL, 'es_ES.utf8')

# date_str_es = '27-Octubre-2020'  # es_ES locale
# datetime_object = datetime.strptime(date_str_es, '%d-%B-%Y')
datetime_object = datetime.strptime(test_date_clean, '%d,%B,%Y')
print(datetime_object.date())

2021-12-29


In [27]:
def make_fms_table(soup, league='', lst=[]):
    locale.setlocale(locale.LC_ALL, 'es_ES.utf8')
    
    fms_tables_titles = soup.select('h3 strong')
    fms_tables = soup.find_all(name='table')
    fms_days_list = fms_tables[0].select('tr')
    
    # Getting the first table of each FMS league and let's convert it into a dataframe:
    
    for i in range(1,len(fms_days_list)):
        object_list = fms_days_list[i].select('td')
        date = ",".join([word for word in str(object_list[1].text).split(" ") if word != 'de'])
        
        try:
            date_clean = datetime.strptime(date, '%d,%B,%Y')
        except:
            date_clean = ""
        
        lst.append({'League': league, '#': object_list[0].text, 'Date': date_clean, 'City': object_list[2].text})
        
    return lst

In [18]:
fms_leagues = ['argentina', 'chile', 'colombia-fechas', 'espana', 'mexico', 'peru-fechas']

In [38]:
def checking_all_fms(fms_list):
    lst = []
    
    for fms in fms_list:
        url_fms = 'https://mundofreestyle.com/calendario-fms-'+fms+'/'
        response = requests.get(url=url_fms)
        soup = BeautifulSoup(response.content)
        lst = make_fms_table(league=fms.split('-')[0].capitalize(), soup=soup, lst=lst)

    return lst

In [39]:
fms_list = checking_all_fms(fms_leagues)
fms_list

[{'League': 'Argentina',
  '#': '1',
  'Date': datetime.datetime(2021, 12, 4, 0, 0),
  'City': 'Mendoza'},
 {'League': 'Argentina',
  '#': '2',
  'Date': datetime.datetime(2021, 12, 29, 0, 0),
  'City': 'Buenos Aires'},
 {'League': 'Argentina',
  '#': '3',
  'Date': datetime.datetime(2022, 4, 30, 0, 0),
  'City': 'La Plata'},
 {'League': 'Argentina',
  '#': '4',
  'Date': datetime.datetime(2022, 6, 18, 0, 0),
  'City': 'Rosario'},
 {'League': 'Argentina',
  '#': '5',
  'Date': datetime.datetime(2022, 8, 27, 0, 0),
  'City': 'Corrientes'},
 {'League': 'Argentina', '#': '6', 'Date': '', 'City': '\xa0'},
 {'League': 'Argentina', '#': '7', 'Date': '', 'City': '\xa0'},
 {'League': 'Argentina', '#': '8', 'Date': '', 'City': '\xa0'},
 {'League': 'Argentina', '#': '9', 'Date': '', 'City': '\xa0'},
 {'League': 'Argentina', '#': '10', 'Date': '', 'City': '\xa0'},
 {'League': 'Argentina', '#': '11', 'Date': '', 'City': '\xa0'},
 {'League': 'Chile',
  '#': '1',
  'Date': datetime.datetime(2022, 2,

In [41]:
fms_df = pd.DataFrame(fms_list)
fms_df

/tmp/ipykernel_3416/3628058372.py:1: FutureWarning: Inferring datetime64[ns] from data containing strings is deprecated and will be removed in a future version. To retain the old behavior explicitly pass Series(data, dtype=datetime64[ns])
  fms_df = pd.DataFrame(fms_list)


,League,#,Date,City
0,Argentina,1,2021-12-04,Mendoza
1,Argentina,2,2021-12-29,Buenos Aires
2,Argentina,3,2022-04-30,La Plata
3,Argentina,4,2022-06-18,Rosario
4,Argentina,5,2022-08-27,Corrientes
5,Argentina,6,NaT,
6,Argentina,7,NaT,
7,Argentina,8,NaT,
8,Argentina,9,NaT,
9,Argentina,10,NaT,


In [42]:
fms_df.to_csv('data/fms_dates.csv', index=False)